# Data Wrangling with Python
### Anthony DeBarros, NICAR 2017 | @anthonydb

##### Among the most unspectacular but necessary tasks in data analysis is reading, transforming, and saving data. You can get a lot done with the Python standard library, and do even more cool things with libraries such as agate. We are going to try sending data back and forth among formats, particularly CSV and JSON, and transforming it along the way.
***

### 1. Open and Transform a CSV using the Python standard library

### Using csv.reader

In [ ]:
# Import the csv module

import csv

In [ ]:
# Open a file and use the reader function to display each line
# file_reader is an iterable reader object
# Each line in the file becomes a Python list

with open('us_counties_2010.csv') as csv_file:
    file_reader = csv.reader(csv_file)
    for row in file_reader:
        print(','.join(row))

In [ ]:
# Because each line is a list, we can call specific elements

with open('us_counties_2010.csv') as csv_file:
    file_reader = csv.reader(csv_file)
    for row in file_reader:
        print(row[0] + ',' + row[1] + ',' + row[9])

In [ ]:
# We also can slice the reader object with itertools.islice() to remove
# the header and just fetch a few rows.

from itertools import islice

with open('us_counties_2010.csv') as csv_file:
    file_reader = csv.reader(csv_file)
    for row in islice(file_reader, 1, 4):
        print(row[0] + ',' + row[1] + ',' + row[9])

### Using csv.DictReader

In [ ]:
# DictReader creates a reader object where each row is an ordered dictionary
# with keys taken from the header row.

with open('us_counties_2010.csv') as csv_file:
    file_reader = csv.DictReader(csv_file)
    for row in islice(file_reader, 1, 4):
        print(row)
        

In [ ]:
# Then, we can pull elements of each line via their key.

with open('us_counties_2010.csv') as csv_file:
    file_reader = csv.DictReader(csv_file)
    for row in islice(file_reader, 1, 4):
        print(row['NAME'] + ',' + row['STUSAB'] + ',' + row['POP100'])
        

### 2. Turning Your CSV into JSON

In [ ]:
# Dictionaries and lists are easily transformed to JSON.


import json
import collections

# Define an empty list of dictionaries. Each dict will hold data on one state.
state_pop_list = []

# Open and read the CSV.
with open('us_counties_2010.csv') as csv_file:
    file_reader = csv.DictReader(csv_file)

    # Turn each row into an ordered dictionary
    for row in islice(file_reader, 1, 4):
        state_dict = collections.OrderedDict()
        state_dict['cty'] = row['NAME']
        state_dict['st'] = row['STUSAB']
        state_dict['pop2010'] = int(row['POP100'])
        # Append the dictionary to the list
        state_pop_list.append(state_dict)

# Use the json library to format the list of dicts as JSON and print.        
print(json.dumps(state_pop_list, indent=4))


In [ ]:
# Write the results to a file.
json_out = json.dumps(state_pop_list)

with open('us_counties_2010.json', 'w') as j:
    j.write(json_out)


### 3. Reading JSON from an API and Transforming to CSV

In [ ]:
import requests

earthquake_geojson_url = 'http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-02-25&endtime=2017-02-26'
r = requests.get(earthquake_geojson_url)
print(r.text)


In [ ]:
response = json.loads(r.text)
print(response['features'][0]['properties']['place'])

In [ ]:
for i in range(0,20):
    print(response['features'][i]['properties']['place'])

In [ ]:
# open and prep file
earthquakes = open('earthquakes.csv', 'w')
quake_writer = csv.writer(earthquakes, delimiter=",")
headers = ['PLACE', 'MAGNITUDE']
quake_writer.writerow(headers)

for i in range(0,20):
    place = response['features'][i]['properties']['place']
    mag = response['features'][i]['properties']['mag']
    quake = (place, mag)
    quake_writer.writerow(quake)
    
earthquakes.close()

### 4. Using the agate Library to Read a CSV, Calculate Stats, and Save as JSON

In [ ]:
# Ignore deprecation warnings if they crop up
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import the agate library
import agate

In [ ]:
# Use agate's from_csv method to load a CSV to a table object
us_counties = agate.Table.from_csv('us_counties_2010.csv')

In [ ]:
# Now we have a table object
us_counties

In [ ]:
# We can view its structure with print()
print(us_counties)

In [ ]:
# Change data types of columns if needed
specified_types = {
    'SUMLEV': agate.Text(),
    'REGION': agate.Text(),
    'DIVISION': agate.Text(),
    'STATE': agate.Text(),
    'COUNTY': agate.Text()    
}
us_counties = agate.Table.from_csv('us_counties_2010.csv', column_types=specified_types)

In [ ]:
print(us_counties)

In [ ]:
# Viewing table contents
us_counties.print_table(max_rows = 10, max_columns = 9)

In [ ]:
# Similar to a SQL SELECT, we can create a new table with a subset of columns
columns = ['STUSAB', 'POP100']
state_pop_table = us_counties.select(columns)

In [ ]:
print(state_pop_table)

In [ ]:
# Let's create a table of states with the total and median population
states = state_pop_table.group_by('STUSAB')
state_pop = states.aggregate([
    ('total_pop', agate.Sum('POP100')),
    ('median_pop', agate.Median('POP100'))
])

In [ ]:
# Take a look at the results
for row in state_pop.rows:
    print(row['STUSAB'], row['total_pop'], row['median_pop'])

In [ ]:
state_pop.print_table(max_rows=51)

In [ ]:
# Save this is a CSV
state_pop.to_csv('state_pop.csv')

In [ ]:
# Or save it as JSON
state_pop.to_json('state_pop.json')

### 5. Using csvkit to Prep for Database Import